# Advanced Lane Finding
## Udacity Self-Driving Car Nanodegree

### By Ethan Peacock
---

### Overview

The goal of this project is to write a software pipeline to indentify the lane
boundaries in a video with a front-facing camera on a car.  The camera
calibration images, test road images, and project videos are available
in this project repository:  https://github.com/udacity/CarND-Advanced-Lane-Lines

Below is a summary of the techniques used in this project.

1. Camera calibration
2. Distortion correction
3. Color/gradient threshold
4. Perspective transform
5. Detect lane lines
6. Determine the lane curvature

---

In [1]:
# Imports
import pickle
import glob
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline
# %matplotlib notebook  #  Enables interactive windows in jupyter notebook

### Compute the camera calibration matrix and distortion coefficients

In [2]:
# # REad in a calibration image
# img = mpimg.imread('./camera_cal/calibration2.jpg')
# plt.imshow(img)

# #  Arrays to store object points and image points from all the images
# objpoints = [] # 3D points in real world space
# imgpoints = [] # 2D points in image plane

# # Prepare object points, like (0,0,0), (1,0,0), (2,0,0), ...., (7,5,0)
# nx = 9 # Number of inside corners of x
# ny = 6 # Number of inside corners of y

# objp = np.zeros((ny*nx,3), np.float32)
# objp[:,:2] = np.mgrid[0:nx,0:6].T.reshape(-1,2) # x, y coordinates

# # Convert image to grayscale
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# # Find the chessboard corners
# ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)


# # If corners are found, add object points, image points
# if ret == True:
#     imgpoints.append(corners)
#     objpoints.append(objp)
    
#     # Draw and display the corners
#     img = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
#     plt.imshow(img)

In [9]:
def cal_undistort(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [13]:
# #  Define name of a picle file
# pickle_file = "wide_dist_pickle.p"
# # Read in and make a list of calibration images
# images = glob.glob('./camera_cal/*')

# # Arrays to store object points and image points from all the imgaes
# objpoints = [] # 3D points in real world space
# imgpoints = [] # 2D points in image plane

# # Prepare object points, like (0,0,0), (1,0,0), (2,0,0), ...., (7,5,0)
# nx = 9 # Number of inside corners of x
# ny = 6 # Number of inside corners of y

# objp = np.zeros((ny*nx,3), np.float32)
# objp[:,:2] = np.mgrid[0:nx,0:6].T.reshape(-1,2) # x, y coordinates

# for fname in images:
#     # Read in each image
#     img = mpimg.imread(fname)
#     # Convert image to grayscale
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     # Find the chessboard corners
#     ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    
#     # If corners are found, add object points, image points
#     if ret == True:
#         print(fname)
#         imgpoints.append(corners)
#         objpoints.append(objp)

#         # Draw and display the corners
#         img = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
#         plt.imshow(img)


# # Save camera calibration result for later use in pickle file    
# dist_pickle = {}
# dist_pickle['objpoints'] = objpoints
# dist_pickle['imgpoints'] = imgpoints
# pickle.dump(dist_pickle, open(pickle_file, 'wb'))